In [1]:
%run data.ipynb    # load data
%run gauss.ipynb

In [2]:
import math
import sympy as sy
from scipy.optimize import fmin
import numpy as np

In [3]:
def gibbs_energy_i(i_name, T):
    x_t = T / 10000
    data = bank_td[i_name]
    phi = data['f1'] + data['f2'] * math.log(x_t) + data['f3'] / (x_t ** 2) + data['f4'] / x_t \
        + data['f5'] * x_t + data['f6'] * x_t ** 2 + data['f7'] * x_t ** 3
    gibbs = data['H'] - phi * T
    return gibbs

def G0(name, T):
    return gibbs_energy_i(name, T)

In [4]:
def calculate_delta_r1(T):
    return 2 * G0('Al', T) + 2 * G0('HCl', T) - 2 * G0('AlCl', T) - G0('H2', T)

def calculate_delta_r2(T):
    return G0('Al', T) + 2 * G0('HCl', T) - G0('AlCl2', T) - G0('H2', T)

def calculate_delta_r3(T):
    return 2 * G0('Al', T) + 6 * G0('HCl', T) - 2 * G0('AlCl3', T) - 3 * G0('H2', T)

In [5]:
Pa = 100000

def K1(T, R = 8.314):
    return math.exp(-1 * calculate_delta_r1(T) / (R * T)) / Pa

def K2(T, R = 8.314):
    return math.exp(-1 * calculate_delta_r2(T) / (R * T))

def K3(T, R = 8.314):
    return math.exp(-1 * calculate_delta_r3(T) / (R * T)) / Pa



In [6]:
def eps(name):
    return math.sqrt(bank_td[name]['epsil'] * bank_td['N2']['epsil'])

def mu(name):
    return 2 * bank_td[name]['mu'] * bank_td['N2']['mu'] / (bank_td[name]['mu'] + bank_td['N2']['mu'])

def omega(name, T):
    return 1.074 * ((T / eps(name)) ** (-0.1604))

def sigma(name):
    return (bank_td[name]['sigma'] + bank_td['N2']['sigma']) / 2

def D(name, T):
    bott = Pa * sigma(name) * omega(name, T) * math.sqrt(mu(name))
    return 0.02628 * (T ** 1.5) / bott

In [7]:
T = 623.15
print(K1(T))
print(K2(T))
print(K3(T))

7.497131271125371e-07
7.221267004644182e-08
4.062199476332757e-49


In [8]:
print(D('AlCl', T))
print(D('AlCl2', T))
print(D('AlCl3', T))
print(D('HCl', T))
print(D('H2', T))

0.000191129931554728
0.00014747988098603198
0.00015245155020045148
0.0002736394738546893
0.0008762807050502889


In [9]:
PG = {
   'AlCl':0,
   'AlCl2':0,
   'AlCl3':0,
   'H2':0,
   'HCl':10000,
   'N2':90000
}

T = 623.15
HCl, AlCl, H2, AlCl2, AlCl3 = sy.symbols('HCl AlCl H2 AlCl2 AlCl3')
X = sy.Matrix([
    HCl**2 - K1(T) * AlCl**2 * H2,
    HCl**2 - K2(T) * AlCl2 * H2,
    HCl**6 - K3(T) * AlCl3**2 * H2**3,
    D('HCl', T) * (PG['HCl'] - HCl) + 2 * D('H2', T) * (PG['H2'] - H2),
    D('AlCl', T) * (PG['AlCl'] - AlCl) + 2 * D('AlCl2', T) * (PG['AlCl2'] - AlCl2) + \
        3 * D('AlCl3', T) * (PG['AlCl3'] - AlCl3) + D('HCl', T) * (PG['HCl'] - HCl)
])
Y = [AlCl, AlCl2, AlCl3, HCl, H2]
X.jacobian(Y)

Matrix([
[-1.49942625422507e-6*AlCl*H2,                       0,                                 0,                 2*HCl,         -7.49713127112537e-7*AlCl**2],
[                           0, -7.22126700464418e-8*H2,                                 0,                 2*HCl,           -7.22126700464418e-8*AlCl2],
[                           0,                       0, -8.12439895266551e-49*AlCl3*H2**3,              6*HCl**5, -1.21865984289983e-48*AlCl3**2*H2**2],
[                           0,                       0,                                 0, -0.000273639473854689,                 -0.00175256141010058],
[       -0.000191129931554728,   -0.000294959761972064,             -0.000457354650601354, -0.000273639473854689,                                    0]])

In [10]:
def new_zip(x, y):
    ans = []
    for i in range(len(x)):
        ans.append((x[i], y[i]))
    return ans

def equalVector(a, b, eps):
    return sum([(a[i] - b[i])**2 for i in range(len(a))]) < eps

def mul_and_add(a, b, c):
    res = []
    for i in range(len(a)):
        res.append(a[i] + b[i] * c)
    return res

def minimumF(Funcs, Vars, xk, pk):
    def r(a):
        res = 0
        xx = new_zip(Vars, mul_and_add(xk, pk, a[0]))
        for i in range(len(xx)):
            res += (Funcs[i].subs(xx)) ** 2
        return res
    return r

def newton_method(funcs, var, x0, eps=1e-20, ITER=50):
    j = funcs.jacobian(var)
    xk1 = sy.zeros(len(x0), 1)
    xk = x0
    iterations = 0
    while iterations < ITER:
        xx = new_zip(var, xk)
        jk = j.subs(xx)
        fk = funcs.subs(xx)
#         dk = [1, 1, 1, 1, 1]
        dk = gaussMethod(jk.tolist(), (fk.T * -1.0).tolist()[0])
        
        alpha = fmin(minimumF(funcs, var, xk, dk), [0], disp=False)[0]
        xk1 = mul_and_add(xk, dk, alpha)
        
        xk, xk1 = xk1, xk
        iterations += 1
    print(iterations)
    return xk

ans = newton_method(X * 1e30, Y, [1, 1, 1, 1, 1])
print(ans)
print(X.subs(new_zip(Y, ans)))

50
[-2.23551418889033, 51.5428297598247, 5950.73870243520, 0.0764849662411599, 1561.35687538572]
Matrix([[2.80442604902453e-11], [3.85089495980396e-5], [2.00196497922386e-7], [2.04474881471128e-9], [2.04535443110857e-9]])
